In [1]:
import pandas as pd
from functools import partial
import sys

sys.path.append("../Handlers")

JSON_WRITE_MODE = "overwrite"

import preprocessing

[nltk_data] Downloading package punkt to E:/nltk...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
csv_file = pd.read_csv("./SpamAssassin.csv")

csv_file.head()

,Subject,Date,From,To,Received,Content-Type,Body,Is_spam
0,Re: New Sequences Window,"Thu, 22 Aug 2002 18:26:25 +0700",['Robert Elz <kre@munnari.OZ.AU>'],['Chris Garrigues <cwg-dated-1030377287.06fa6d...,['from localhost (localhost [127.0.0.1])\tby p...,text/plain,"Date: Wed, 21 Aug 2002 10:54:46 -05...",0
1,[zzzzteana] RE: Alexander,"Thu, 22 Aug 2002 12:46:18 +0100",['Steve Burt <Steve_Burt@cursor-system.com>'],"['""\'zzzzteana@yahoogroups.com\'"" <zzzzteana@y...",['from localhost (localhost [127.0.0.1])\tby p...,text/plain,"Martin A posted:\nTassos Papadopoulos, the Gre...",0
2,[zzzzteana] Moscow bomber,"Thu, 22 Aug 2002 13:52:38 +0100",['Tim Chapman <timc@2ubh.com>'],['zzzzteana <zzzzteana@yahoogroups.com>'],['from localhost (localhost [127.0.0.1])\tby p...,text/plain,Man Threatens Explosion In Moscow \n\nThursday...,0
3,[IRR] Klez: The Virus That Won't Die,"Thu, 22 Aug 2002 09:15:25 -0400",['Monty Solomon <monty@roscom.com>'],['undisclosed-recipient:;'],['from localhost (localhost [127.0.0.1])\tby p...,text/plain,Klez: The Virus That Won't Die\n \nAlready the...,0
4,Re: Insert signature,"Thu, 22 Aug 2002 23:36:32 +1000",['Tony Nugent <tony@linuxworks.com.au>'],['Exmh Users Mailing List <exmh-users@example....,['from localhost (localhost [127.0.0.1])\tby p...,text/plain,"On Wed Aug 21 2002 at 15:46, Ulises Ponce wrot...",0


In [3]:
preprocession = partial(
    preprocessing.preprocess_text,
    remove_numbers=True
)

preprocessed_data = csv_file["Body"].apply(preprocession)
preprocessed_data

0       [date, DATE, TIME, chris, garrigues, messageid...
1       [martin, posted, tassos, papadopoulos, greek, ...
2       [man, threatens, explosion, moscow, thursday, ...
3       [klez, virus, wont, die, already, prolific, vi...
4       [wed, DATE, TIME, ulises, ponce, wrote, hi, co...
                              ...                        
4193    [preferred, nonsmoker, doctor, ordered, case, ...
4194    [dear, subscriber, could, show, way, get, visi...
4195    [midsummer, customer, appreciation, sale, expr...
4196    [attnsirmadan, strictly, confidential, pleased...
4197    [mv, efaeefcdfdcbaPHONENUMefaeefcdfdcba, mv, e...
Name: Body, Length: 4198, dtype: object

## Stemming + CountVectorizer

In [4]:
from traintest import ClassificationModel, models, add_to_json_array

metric_results = []

def train_and_evaluate_model(X, y, dataset_name, m=None, mode="grid"):
    print(f"{dataset_name} classification report")
    print("=========================================")
    if m is None:
        for model in models:
            classification_model = ClassificationModel(model, dataset_name)
            classification_model.train_with_finding_best_parameters(X, y, save_model=True, mode=mode, n_jobs=6, cv=4)
            classification_model.get_best_estimator(put_right_in_the_model=True)
            print(f"{model.__class__.__name__} classification report")
            metrics = classification_model.evaluate(detailed=True)
            metric_results.append(metrics)
            print(metrics)
            print("\n")
    else:
        classification_model = ClassificationModel(m, dataset_name)
        classification_model.train_with_finding_best_parameters(X, y, save_model=True, mode=mode, n_jobs=6, cv=4)
        classification_model.get_best_estimator(put_right_in_the_model=True)
        print(f"{m.__class__.__name__} classification report")
        metrics = classification_model.evaluate(detailed=True)
        metric_results.append(metrics)
        print(metrics)
        print("\n")

In [5]:
spam_assassin_stemming = preprocessed_data.apply(preprocessing.stemming)
spam_assassin_stemming_countvec = preprocessing.vectorizing(spam_assassin_stemming, "countvectorizer")

In [6]:
train_and_evaluate_model(spam_assassin_stemming_countvec, csv_file["Is_spam"], "spam_assassin_stemming_countvec")

spam_assassin_stemming_countvec classification report
The best estimator for SVM of dataset spam_assassin_stemming_countvec is: 
SVC(C=0.1, kernel='linear')
SVC classification report
{'dataset': 'spam_assassin_stemming_countvec', 'model': 'SVM', 'type': 'grid_search', 'metrics': {'accuracy': 0.9821428571428571, 'weighted_precision': 0.9821278938925998, 'wighted_recall': 0.9821428571428571, 'weighted_f1': 0.9820944920324062, 'macro_precision': 0.981649069884364, 'macro_recall': 0.9764588859416445, 'macro_f1': 0.9789991249635401, 'roc_auc': 0.9764588859416448}, 'confusion_matrix': array([[575,   5],
       [ 10, 250]], dtype=int64), 'best_parameters': {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}, 'best_score': 0.9687311992735116}


The best estimator for Multinomial Naive Bayes of dataset spam_assassin_stemming_countvec is: 
MultinomialNB(fit_prior=False)
MultinomialNB classification report
{'dataset': 'spam_assassin_stemming_countvec', 'model': 'Multinomial Naive Bayes', 'type': 'gr

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise ValueError(
Valu

The best estimator for Logistic Regression of dataset spam_assassin_stemming_countvec is: 
LogisticRegression(C=1, solver='liblinear')
LogisticRegression classification report
{'dataset': 'spam_assassin_stemming_countvec', 'model': 'Logistic Regression', 'type': 'grid_search', 'metrics': {'accuracy': 0.9785714285714285, 'weighted_precision': 0.9785930245113919, 'wighted_recall': 0.9785714285714285, 'weighted_f1': 0.9784774033757339, 'macro_precision': 0.9790249433106576, 'macro_recall': 0.9706896551724138, 'macro_f1': 0.9747163955479452, 'roc_auc': 0.9706896551724139}, 'confusion_matrix': array([[575,   5],
       [ 13, 247]], dtype=int64), 'best_parameters': {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}, 'best_score': 0.9740936630909813}


The best estimator for K-nearest Neighbors of dataset spam_assassin_stemming_countvec is: 
KNeighborsClassifier(n_neighbors=3, weights='distance')
KNeighborsClassifier classification report
{'dataset': 'spam_assassin_stemming_countvec', 'model': 

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
208 fits failed out of a total of 280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
208 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 892, in fit
    self._more_validate_params()
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 149, in _more_validate_params
    raise ValueError("eta0

The best estimator for Stochastic Gradient Descent of dataset spam_assassin_stemming_countvec is: 
SGDClassifier(alpha=0.01, loss='log_loss')
SGDClassifier classification report
{'dataset': 'spam_assassin_stemming_countvec', 'model': 'Stochastic Gradient Descent', 'type': 'grid_search', 'metrics': {'accuracy': 0.9666666666666667, 'weighted_precision': 0.9672222222222221, 'wighted_recall': 0.9666666666666667, 'weighted_f1': 0.9662824858757062, 'macro_precision': 0.9716666666666667, 'macro_recall': 0.9503978779840849, 'macro_f1': 0.960135593220339, 'roc_auc': 0.9503978779840848}, 'confusion_matrix': array([[576,   4],
       [ 24, 236]], dtype=int64), 'best_parameters': {'penalty': 'l2', 'loss': 'log_loss', 'learning_rate': 'optimal', 'alpha': 0.01}, 'best_score': 0.9708180799137295}


The best estimator for Gradient Boosting of dataset spam_assassin_stemming_countvec is: 
GradientBoostingClassifier(max_depth=7, min_samples_leaf=4, n_estimators=200)
GradientBoostingClassifier classificat

## Stemming + TF-IDF Vectorizer

In [7]:
spam_assassin_stemming_tfidf = preprocessing.vectorizing(spam_assassin_stemming, "tfidf")

In [8]:
train_and_evaluate_model(spam_assassin_stemming_tfidf, csv_file["Is_spam"], "spam_assassin_stemmed_tfidf")

spam_assassin_stemmed_tfidf classification report
The best estimator for SVM of dataset spam_assassin_stemmed_tfidf is: 
SVC(C=10, kernel='linear')
SVC classification report
{'dataset': 'spam_assassin_stemmed_tfidf', 'model': 'SVM', 'type': 'grid_search', 'metrics': {'accuracy': 0.9773809523809524, 'weighted_precision': 0.9773371649518776, 'wighted_recall': 0.9773809523809524, 'weighted_f1': 0.9773444998376851, 'macro_precision': 0.9750018354012187, 'macro_recall': 0.971949602122016, 'macro_f1': 0.9734562285558426, 'roc_auc': 0.971949602122016}, 'confusion_matrix': array([[572,   8],
       [ 11, 249]], dtype=int64), 'best_parameters': {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}, 'best_score': 0.9734962966116125}


The best estimator for Multinomial Naive Bayes of dataset spam_assassin_stemmed_tfidf is: 
MultinomialNB(alpha=0.1, fit_prior=False)
MultinomialNB classification report
{'dataset': 'spam_assassin_stemmed_tfidf', 'model': 'Multinomial Naive Bayes', 'type': 'grid_search', 

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise ValueError(
Valu

The best estimator for Logistic Regression of dataset spam_assassin_stemmed_tfidf is: 
LogisticRegression(C=0.1, penalty=None, solver='saga')
LogisticRegression classification report
{'dataset': 'spam_assassin_stemmed_tfidf', 'model': 'Logistic Regression', 'type': 'grid_search', 'metrics': {'accuracy': 0.9845238095238096, 'weighted_precision': 0.9845220139337786, 'wighted_recall': 0.9845238095238096, 'weighted_f1': 0.9844818930947521, 'macro_precision': 0.9844645550527904, 'macro_recall': 0.9792440318302387, 'macro_f1': 0.9817992416350682, 'roc_auc': 0.9792440318302388}, 'confusion_matrix': array([[576,   4],
       [  9, 251]], dtype=int64), 'best_parameters': {'C': 0.1, 'penalty': None, 'solver': 'saga'}, 'best_score': 0.979155315284636}


The best estimator for K-nearest Neighbors of dataset spam_assassin_stemmed_tfidf is: 
KNeighborsClassifier(n_neighbors=3, weights='distance')
KNeighborsClassifier classification report
{'dataset': 'spam_assassin_stemmed_tfidf', 'model': 'K-neares

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
208 fits failed out of a total of 280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
208 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 892, in fit
    self._more_validate_params()
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 149, in _more_validate_params
    raise ValueError("eta0

The best estimator for Stochastic Gradient Descent of dataset spam_assassin_stemmed_tfidf is: 
SGDClassifier()
SGDClassifier classification report
{'dataset': 'spam_assassin_stemmed_tfidf', 'model': 'Stochastic Gradient Descent', 'type': 'grid_search', 'metrics': {'accuracy': 0.9761904761904762, 'weighted_precision': 0.9767827692728878, 'wighted_recall': 0.9761904761904762, 'weighted_f1': 0.9759455185966459, 'macro_precision': 0.9820475966721026, 'macro_recall': 0.9625994694960213, 'macro_f1': 0.9715907169285507, 'roc_auc': 0.9625994694960213}, 'confusion_matrix': array([[579,   1],
       [ 19, 241]], dtype=int64), 'best_parameters': {'penalty': 'l2', 'loss': 'hinge', 'learning_rate': 'optimal', 'alpha': 0.0001}, 'best_score': 0.97260698677564}


The best estimator for Gradient Boosting of dataset spam_assassin_stemmed_tfidf is: 
GradientBoostingClassifier(max_depth=7, min_samples_leaf=4, n_estimators=200)
GradientBoostingClassifier classification report
{'dataset': 'spam_assassin_ste

## Lemmatizing + CountVectorizer

In [9]:
spam_assassin_lemmatizing = preprocessed_data.apply(preprocessing.lemmatizing)
spam_assassin_lemmatizing_countvec = preprocessing.vectorizing(spam_assassin_lemmatizing, "countvectorizer")

In [10]:
train_and_evaluate_model(spam_assassin_lemmatizing_countvec, csv_file["Is_spam"], "spam_assassin_lemmatized_countvec")

spam_assassin_lemmatized_countvec classification report
The best estimator for SVM of dataset spam_assassin_lemmatized_countvec is: 
SVC(C=0.1, kernel='linear')
SVC classification report
{'dataset': 'spam_assassin_lemmatized_countvec', 'model': 'SVM', 'type': 'grid_search', 'metrics': {'accuracy': 0.9845238095238096, 'weighted_precision': 0.9845669600872919, 'wighted_recall': 0.9845238095238096, 'weighted_f1': 0.9844646367895364, 'macro_precision': 0.985553258681175, 'macro_recall': 0.9781830238726791, 'macro_f1': 0.9817595975084814, 'roc_auc': 0.9781830238726792}, 'confusion_matrix': array([[577,   3],
       [ 10, 250]], dtype=int64), 'best_parameters': {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}, 'best_score': 0.9726041489301323}


The best estimator for Multinomial Naive Bayes of dataset spam_assassin_lemmatized_countvec is: 
MultinomialNB(alpha=0.5, fit_prior=False)
MultinomialNB classification report
{'dataset': 'spam_assassin_lemmatized_countvec', 'model': 'Multinomial Naiv

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise ValueError(
Valu

The best estimator for Logistic Regression of dataset spam_assassin_lemmatized_countvec is: 
LogisticRegression(C=1, solver='liblinear')
LogisticRegression classification report
{'dataset': 'spam_assassin_lemmatized_countvec', 'model': 'Logistic Regression', 'type': 'grid_search', 'metrics': {'accuracy': 0.9845238095238096, 'weighted_precision': 0.9845669600872919, 'wighted_recall': 0.9845238095238096, 'weighted_f1': 0.9844646367895364, 'macro_precision': 0.985553258681175, 'macro_recall': 0.9781830238726791, 'macro_f1': 0.9817595975084814, 'roc_auc': 0.9781830238726792}, 'confusion_matrix': array([[577,   3],
       [ 10, 250]], dtype=int64), 'best_parameters': {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}, 'best_score': 0.9755814035983881}


The best estimator for K-nearest Neighbors of dataset spam_assassin_lemmatized_countvec is: 
KNeighborsClassifier(n_neighbors=3, weights='distance')
KNeighborsClassifier classification report
{'dataset': 'spam_assassin_lemmatized_countvec', 'm

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
208 fits failed out of a total of 280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
208 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 892, in fit
    self._more_validate_params()
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 149, in _more_validate_params
    raise ValueError("eta0

The best estimator for Stochastic Gradient Descent of dataset spam_assassin_lemmatized_countvec is: 
SGDClassifier(alpha=0.01)
SGDClassifier classification report
{'dataset': 'spam_assassin_lemmatized_countvec', 'model': 'Stochastic Gradient Descent', 'type': 'grid_search', 'metrics': {'accuracy': 0.9761904761904762, 'weighted_precision': 0.9767827692728878, 'wighted_recall': 0.9761904761904762, 'weighted_f1': 0.9759455185966459, 'macro_precision': 0.9820475966721026, 'macro_recall': 0.9625994694960213, 'macro_f1': 0.9715907169285507, 'roc_auc': 0.9625994694960213}, 'confusion_matrix': array([[579,   1],
       [ 19, 241]], dtype=int64), 'best_parameters': {'penalty': 'l2', 'loss': 'hinge', 'learning_rate': 'optimal', 'alpha': 0.01}, 'best_score': 0.9758786679153187}


The best estimator for Gradient Boosting of dataset spam_assassin_lemmatized_countvec is: 
GradientBoostingClassifier(max_depth=7, min_samples_leaf=4, n_estimators=200)
GradientBoostingClassifier classification report
{'

## Lemmatizing + TF-IDF Vectorizer

In [11]:
spam_assassin_lemmatizing_tfidf = preprocessing.vectorizing(spam_assassin_lemmatizing, "tfidf")

In [12]:
train_and_evaluate_model(spam_assassin_lemmatizing_tfidf, csv_file["Is_spam"], "spam_assassin_lemmatized_tfidf")

spam_assassin_lemmatized_tfidf classification report
The best estimator for SVM of dataset spam_assassin_lemmatized_tfidf is: 
SVC(C=10, kernel='sigmoid')
SVC classification report
{'dataset': 'spam_assassin_lemmatized_tfidf', 'model': 'SVM', 'type': 'grid_search', 'metrics': {'accuracy': 0.975, 'weighted_precision': 0.9749765748044577, 'wighted_recall': 0.975, 'weighted_f1': 0.9749866818021138, 'macro_precision': 0.9712285435177002, 'macro_recall': 0.9702254641909814, 'macro_f1': 0.9707248584785888, 'roc_auc': 0.9702254641909814}, 'confusion_matrix': array([[570,  10],
       [ 11, 249]], dtype=int64), 'best_parameters': {'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}, 'best_score': 0.9758779584539418}


The best estimator for Multinomial Naive Bayes of dataset spam_assassin_lemmatized_tfidf is: 
MultinomialNB(alpha=0.1, fit_prior=False)
MultinomialNB classification report
{'dataset': 'spam_assassin_lemmatized_tfidf', 'model': 'Multinomial Naive Bayes', 'type': 'grid_search', 'metric

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 144.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise ValueError(
Valu

The best estimator for Logistic Regression of dataset spam_assassin_lemmatized_tfidf is: 
LogisticRegression(C=1, penalty=None, solver='saga')
LogisticRegression classification report
{'dataset': 'spam_assassin_lemmatized_tfidf', 'model': 'Logistic Regression', 'type': 'grid_search', 'metrics': {'accuracy': 0.9904761904761905, 'weighted_precision': 0.9905333932234581, 'wighted_recall': 0.9904761904761905, 'weighted_f1': 0.9904451078373697, 'macro_precision': 0.9920587998172583, 'macro_recall': 0.9856763925729444, 'macro_f1': 0.9887873671002663, 'roc_auc': 0.9856763925729444}, 'confusion_matrix': array([[579,   1],
       [  7, 253]], dtype=int64), 'best_parameters': {'C': 1, 'penalty': None, 'solver': 'saga'}, 'best_score': 0.9809413843010387}


The best estimator for K-nearest Neighbors of dataset spam_assassin_lemmatized_tfidf is: 
KNeighborsClassifier(n_neighbors=3, weights='distance')
KNeighborsClassifier classification report
{'dataset': 'spam_assassin_lemmatized_tfidf', 'model': 

e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
208 fits failed out of a total of 280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
208 fits failed with the following error:
Traceback (most recent call last):
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 892, in fit
    self._more_validate_params()
  File "e:\Python Tests\AI\.venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 149, in _more_validate_params
    raise ValueError("eta0

The best estimator for Stochastic Gradient Descent of dataset spam_assassin_lemmatized_tfidf is: 
SGDClassifier()
SGDClassifier classification report
{'dataset': 'spam_assassin_lemmatized_tfidf', 'model': 'Stochastic Gradient Descent', 'type': 'grid_search', 'metrics': {'accuracy': 0.9773809523809524, 'weighted_precision': 0.978098417998251, 'wighted_recall': 0.9773809523809524, 'weighted_f1': 0.9771343052414496, 'macro_precision': 0.9841402337228715, 'macro_recall': 0.9634615384615385, 'macro_f1': 0.9729802481550893, 'roc_auc': 0.9634615384615385}, 'confusion_matrix': array([[580,   0],
       [ 19, 241]], dtype=int64), 'best_parameters': {'penalty': 'l2', 'loss': 'hinge', 'learning_rate': 'optimal', 'alpha': 0.0001}, 'best_score': 0.9743930557920426}


The best estimator for Gradient Boosting of dataset spam_assassin_lemmatized_tfidf is: 
GradientBoostingClassifier(max_depth=7, min_samples_leaf=4, min_samples_split=5,
                           n_estimators=200)
GradientBoostingClass

In [13]:
print(metric_results)

def convert(dic):
    dic["confusion_matrix"] = dic["confusion_matrix"].tolist()
    return dic

metric_results = list(map(convert, metric_results))

add_to_json_array("./spam_assassin_grid_classification_report.json", metric_results, mode="overwrite")

[{'dataset': 'spam_assassin_stemming_countvec', 'model': 'SVM', 'type': 'grid_search', 'metrics': {'accuracy': 0.9821428571428571, 'weighted_precision': 0.9821278938925998, 'wighted_recall': 0.9821428571428571, 'weighted_f1': 0.9820944920324062, 'macro_precision': 0.981649069884364, 'macro_recall': 0.9764588859416445, 'macro_f1': 0.9789991249635401, 'roc_auc': 0.9764588859416448}, 'confusion_matrix': array([[575,   5],
       [ 10, 250]], dtype=int64), 'best_parameters': {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}, 'best_score': 0.9687311992735116}, {'dataset': 'spam_assassin_stemming_countvec', 'model': 'Multinomial Naive Bayes', 'type': 'grid_search', 'metrics': {'accuracy': 0.969047619047619, 'weighted_precision': 0.9691028874496616, 'wighted_recall': 0.969047619047619, 'weighted_f1': 0.9688404990331839, 'macro_precision': 0.9698397994768962, 'macro_recall': 0.9574270557029179, 'macro_f1': 0.9633172986482492, 'roc_auc': 0.9574270557029178}, 'confusion_matrix': array([[573,   7]